In [ ]:
!ls

sample_data


#External Databases

In [1]:
! pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-x_9zp_z7
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-x_9zp_z7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 9.0 MB/s 
     |████████████████████████████████| 6.5 MB 45.2 MB/s 
     |████████████████████████████████| 67 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 66.4 MB/s 
  Created wheel for transformers: filename=transformers-4.18.0.dev0-py3-none-any.whl size=3835851 sha256=cf8f9b2c73ff8a9aa154194feb583c59d65b9c36cb3a4b9b870108ea3e45f07f
  Stored in directory: /tmp/pip-ephem-wheel-cache-njbw699c/wheels/90/a5/44/6bcd83827c8a60628c5ad602f429cd5076bcce5f2a90054947
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfu

In [2]:
! pip install ray[tune]

     |████████████████████████████████| 52.7 MB 62 kB/s 
     |████████████████████████████████| 4.1 MB 52.7 MB/s 
     |████████████████████████████████| 175 kB 7.1 MB/s 
     |████████████████████████████████| 125 kB 55.7 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.44.0
    Uninstalling grpcio-1.44.0:
      Successfully uninstalled grpcio-1.44.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [3]:
!pip install datasets

     |████████████████████████████████| 312 kB 8.1 MB/s 
     |████████████████████████████████| 134 kB 67.4 MB/s 
     |████████████████████████████████| 1.1 MB 52.7 MB/s 
     |████████████████████████████████| 212 kB 52.3 MB/s 
     |████████████████████████████████| 127 kB 71.3 MB/s 
     |████████████████████████████████| 271 kB 42.3 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 144 kB 50.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
!pip install seqeval

     |████████████████████████████████| 43 kB 1.4 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=6b9874e5ed549c9949d3c1476544b5a01defc21d2779cca015b1100b6efb7dc4
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


#Libraries

In [5]:
from google.colab import files

import os
import re
import json
import string

import torch
import torch.nn as nn

import numpy as np
#import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras import layers

from datasets import load_metric
from sklearn.metrics import f1_score, precision_score, recall_score
from seqeval.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support as prfs

import tensorflow_hub as hub
from keras import backend as K

import transformers
print(transformers.__version__)
from transformers import TrainingArguments
from transformers import Trainer
from transformers.trainer_utils import EvalLoopOutput

from transformers import AutoTokenizer, AutoModel
BertTokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
BertEmbModel = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')

4.18.0.dev0


Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#Loading Dataset

In [6]:
uploaded = files.upload()

Saving test_500_v2.conll to test_500_v2.conll
Saving train_1500_v2.conll to train_1500_v2.conll


In [7]:
class TDMSciDataset(torch.utils.data.Dataset):
  def __init__(self, raw_x, raw_y, max_length=250):
    self.raw_x = raw_x
    self.raw_y = raw_y
    
    self.max_length = max_length

  def tokenize_and_preserve_labels(self, sentence, text_labels):
    """
    The tokenizer can split single words into multiple tokens - this breaks
    the labels, so we need to keep track of this!
    """
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = BertTokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

  def __getitem__(self, idx):
    tokens = self.raw_x[idx]
    labels = np.zeros((len(tokens)))
    for i, label in enumerate(self.raw_y[idx]):
      labels[i] = label

    # This could be moved to __init__ to save time?
    tokens, labels = self.tokenize_and_preserve_labels(tokens, labels)

    # Convert each token to an id number 
    input_ids = BertTokenizer.convert_tokens_to_ids(tokens)
    
    # add and adjust for special tokens
    input_ids = [BertTokenizer.cls_token_id] + input_ids + [BertTokenizer.sep_token_id]  
    labels = [0] + labels + [0]

    # Pad inputs
    input_ids = torch.tensor(np.pad(input_ids, [0, self.max_length-len(input_ids)]))
    labels = torch.tensor(np.pad(labels, [0, self.max_length-len(labels)], constant_values=-100))

    attention_mask = torch.tensor([int(i != 0) for i in input_ids])

    #return {input_ids, labels}
    #return {'input_ids': [input_ids], 'labels': labels}
    return {'input_ids': [input_ids], 'attention_mask': [attention_mask], 'labels': labels}

  def __len__(self):
    return len(self.raw_y)


In [8]:
# Entity maps
def Entity2ID(Sc_Entity):
  return {
        'TASK': 1,
        'DATASET': 2,
        'METRIC': 3,
        'None': 0,
        'O': 0,
    }[Sc_Entity]

def ID2Entity(Sc_Entity):
  return {
        1: 'TASK',
        2: 'DATASET',
        3: 'METRIC',
        0: 'None',
    }[Sc_Entity]

In [9]:
# Load raw data train
raw_x = []
raw_y = []

cur_x = []
cur_y = []
with open('train_1500_v2.conll', 'r') as f:
  for line in f:
    if len(line.strip()) <= 3:
      raw_x.append(cur_x)
      raw_y.append(cur_y)
      cur_x = []
      cur_y = []
    else:
      line = line.strip().split('\t')
      cur_x.append(line[0])
      cur_y.append(Entity2ID(line[-1].split('-')[-1]))

print(raw_x[10])
print(raw_y[10])

# Create actual dataset
train = TDMSciDataset(raw_x, raw_y)

#print(train[0])

['The', 'results', 'show', 'that', 'proposed', 'shallow', 'representations', 'of', 'sentence', 'structure', 'are', 'robust', 'to', 'reductions', 'in', 'parsing', 'accuracy', ',', 'and', 'that', 'the', 'contribution', 'of', 'alternative', 'representations', 'of', 'sentence', 'structure', 'to', 'successful', 'semantic', 'role', 'labeling', 'varies', 'with', 'the', 'integrity', 'of', 'the', 'parsing', 'and', 'argument', '-', 'identification', 'stages', '.', '.']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0]


In [10]:
# Load raw data test
raw_x = []
raw_y = []

cur_x = []
cur_y = []
with open('test_500_v2.conll', 'r') as f:
  for line in f:
    if len(line.strip()) <= 3:
      raw_x.append(cur_x)
      raw_y.append(cur_y)
      cur_x = []
      cur_y = []
    else:
      line = line.strip().split('\t')
      cur_x.append(line[0])
      cur_y.append(Entity2ID(line[-1].split('-')[-1]))

print(raw_x[10])
print(raw_y[10])

# Create actual dataset
test = TDMSciDataset(raw_x, raw_y)

#print(test[0])

['We', 'evaluate', 'our', 'approach', 'on', 'three', 'datasets', ':', 'TriviaQA', 'unfiltered', '(', ',', 'a', 'dataset', 'of', 'questions', 'from', 'trivia', 'databases', 'paired', 'with', 'documents', 'found', 'by', 'completing', 'a', 'web', 'search', 'of', 'the', 'questions', ';', 'TriviaQA', 'web', ',', 'a', 'dataset', 'derived', 'from', 'TriviaQA', 'unfiltered', 'by', 'treating', 'each', 'question', 'document', 'pair', 'where', 'the', 'document', 'contains', 'the', 'question', 'answer', 'as', 'an', 'individual', 'training', 'point', ';', 'and', 'SQuAD', '(', ',', 'a', 'collection', 'of', 'Wikipedia', 'articles', 'and', 'crowdsourced', 'questions', '.']
[0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [11]:
len(test), len(train)

(486, 1522)

In [12]:
#train[1]

In [13]:
local_hp_train = []
local_hp_val = []
for x in range(500):
  local_hp_train.append(train[x])

for x in range(180):
  local_hp_val.append(test[x])

#print(len(local_hp_train))
#print(local_hp_train[1])

# Pytorch Model Definition

In [14]:
class NerModel(nn.Module):
  def __init__(self, b_embeddings, emb_dims=768, ff_dims=14, out_dims=4):
    super(NerModel, self).__init__()
    self.sci_embeddings = b_embeddings
    self.embd_dropout = nn.Dropout(0.1)
    self.ff_dropout = nn.Dropout(0.1)
    self.ff = nn.Linear(emb_dims, ff_dims)
    self.tanh = nn.Tanh()
    self.lstm = nn.LSTM(768, 100, 1, bidirectional=True)
    self.lstm_drop = nn.Dropout(0.4)
    self.ff = nn.Linear(200, 14)
    self.ff_act = nn.ReLU()
    self.classifier = nn.Linear(ff_dims, out_dims)
  def forward(self, **inputs):
    embds = self.sci_embeddings(**inputs)['last_hidden_state']
    out = self.embd_dropout(embds)
    out, _ = self.lstm(out)
    out = self.tanh(out)
    out = self.lstm_drop(out)
    out = self.ff(out)
    out = self.ff_act(out)
    out = self.ff_dropout(out)
    out = self.classifier(out)
    return out


# Metrics and Configs

In [15]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    #print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

import math

def load_param():
  for n, v in best_run.hyperparameters.items():
    if n == 'seed':
      setattr(trainer.args, n, math.ceil(v))
      print(n, math.ceil(v))
    else:
      setattr(trainer.args, n, v)
      print(n, v)   

In [16]:
batch_size = 4
training_args = TrainingArguments(
    "trained_scibert_ner_model", # output dir
    learning_rate=1e-5, 
    num_train_epochs=10, 
    dataloader_drop_last=True,
    per_device_eval_batch_size=batch_size, 
    per_device_train_batch_size=batch_size,
    save_steps=len(train) // batch_size,
    eval_steps=len(train) // batch_size,
    lr_scheduler_type='cosine',
    evaluation_strategy='steps'
    )
#print(training_args)

In [17]:
def collator(batch):
  out =  {
      'input_ids': torch.stack([(x['input_ids'][0]) for x in batch]),
      'attention_mask': torch.stack([x['attention_mask'][0] for x in batch]),
      'labels': torch.stack([x['labels'].clone().detach() for x in batch])
  }
  return out

In [18]:
import torch.nn.functional as F

class FocalLoss(nn.modules.loss._WeightedLoss):
    def __init__(self, weight, gamma, reduction='mean'):
        super(FocalLoss, self).__init__(weight,reduction=reduction)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights

    def forward(self, input, target):
        ce_loss = F.cross_entropy(input, target, ignore_index=-100, reduction=self.reduction, weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

In [19]:
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_labels=4):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs

        weights = torch.tensor([0.85, 1.55, 1.65, 1.85]).cuda()  # The no-class label has too many examples, we need to weight the loss - this probably needs further tuning 
        gamma=5
        loss_fct = FocalLoss(weight=weights, gamma=gamma)
        loss = loss_fct(logits.view(-1, num_labels), labels.long().view(-1))
        return (loss, outputs) if return_outputs else loss

    def evaluation_loop(self, dataloader, description, prediction_loss_only=None, ignore_keys=None, metric_key_prefix="eval", num_labels=4):
      args = self.args
      prediction_loss_only = prediction_loss_only if prediction_loss_only is not None else args.prediction_loss_only

      self.model.eval()

      all_losses = []
      all_preds = []
      all_labels = []
      for step, sample in enumerate(dataloader):
        for i in range(0, len(sample['labels'])):
          inputs = {}
          inputs['input_ids'] = torch.stack([sample['input_ids'][i].cuda()])
          inputs['attention_mask'] = torch.stack([sample['attention_mask'][i].cuda()])
          inputs['labels'] = torch.stack([sample['labels'][i].cuda()])
          labels = inputs['labels'][0].cpu().numpy()
          
          (loss, logits) = self.compute_loss(self.model, inputs, return_outputs=True)
          logits = logits[0].cpu().detach().numpy()
          preds = np.argmax(nn.Softmax(dim=-1)(torch.tensor(logits)).numpy(), axis=-1)

          all_losses = np.concatenate((all_losses, [loss.detach().cpu().numpy()]), axis=0)

          preds = preds[labels != -100]
          labels = labels[labels != -100]
          all_preds = np.concatenate((all_preds, preds))
          all_labels = np.concatenate((all_labels, labels))

      metrics = {}
      metrics['macro_f1'] = f1_score(all_labels, all_preds, average='macro')
      metrics['macro_precision'] = precision_score(all_labels, all_preds, average='macro', zero_division=0)
      metrics['macro_recall'] = recall_score(all_labels, all_preds, average='macro', zero_division=0)
      metrics['micro_f1'] = f1_score(all_labels, all_preds, average='micro')
      metrics['micro_precision'] = precision_score(all_labels, all_preds, average='micro', zero_division=0)
      metrics['micro_recall'] = recall_score(all_labels, all_preds, average='micro', zero_division=0)

      metrics['macro_f1_no_o'] = f1_score(all_labels, all_preds, average='macro', labels=[1, 2, 3])
      metrics['macro_precision_no_o'] = precision_score(all_labels, all_preds, average='macro', labels=[1, 2, 3], zero_division=0)
      metrics['macro_recall_no_o'] = recall_score(all_labels, all_preds, average='macro', labels=[1, 2, 3], zero_division=0)
      metrics['micro_f1_no_o'] = f1_score(all_labels, all_preds, average='micro', labels=[1, 2, 3])
      metrics['micro_precision_no_o'] = precision_score(all_labels, all_preds, average='micro', labels=[1, 2, 3], zero_division=0)
      metrics['micro_recall_no_o'] = recall_score(all_labels, all_preds, average='micro', labels=[1, 2, 3], zero_division=0)

      for key in list(metrics.keys()):
        if not key.startswith(metric_key_prefix):
          metrics[metric_key_prefix + '_' + key] = metrics.pop(key)
      
      metrics[metric_key_prefix + '_loss'] = all_losses.mean().item()

      return EvalLoopOutput(predictions=all_preds, label_ids=all_labels, metrics=metrics, num_samples=len(dataloader))

In [20]:
def my_hp_space_ray(trial):
    from ray import tune

    return {
        "learning_rate": tune.loguniform(1e-6, 1e-4),
        "num_train_epochs": tune.choice(range(8, 20)),
        "weight_decay": tune.uniform(0.0, 0.5),
        "per_device_train_batch_size": tune.choice([4, 8, 16]),  #<16 definetly not working
    }

# Hyperparameter Tuning

In [ ]:
def model_init():
    x = NerModel(BertEmbModel)
    x.sci_embeddings.requires_grad = False
    return x

trainer = MultilabelTrainer(model_init=model_init,
                            args=training_args,
                            train_dataset=local_hp_train,
                            eval_dataset=local_hp_val,
                            data_collator=collator  # defines how to merge data into batches, using the collator function above
                            )

In [ ]:
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

best_run = trainer.hyperparameter_search(backend="ray", 
                                         resources_per_trial={"gpu": 1, "cpu": 0},
                                         n_trials=16, 
                                         direction="maximize", 
                                         hp_space=my_hp_space_ray,
                                         search_alg=HyperOptSearch(metric='eval_micro_recall_no_o', mode="max"),  #'eval_*f1_micro'
                                         scheduler=ASHAScheduler(metric='eval_micro_recall_no_o', mode="max")
                                         )

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
best_run

BestRun(run_id='63b4738c', objective=9.156609578992219, hyperparameters={'learning_rate': 7.648631456296879e-06, 'num_train_epochs': 19, 'weight_decay': 0.44441579284898636, 'per_device_train_batch_size': 8})

# Cossvalidation

In [ ]:
#import tensorflow as tf
from torch.utils.data import DataLoader, ConcatDataset
from sklearn.model_selection import KFold
from torch import nn
from transformers import Trainer

#print('GPU detected:', tf.config.list_physical_devices('GPU'))
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True)
results = np.zeros(5)
resultss = np.zeros(5)
dataset = ConcatDataset([train, test])



In [ ]:
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    
    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
    
    # Sample elements randomly from a given list of ids, no replacement.
    #train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    #test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    local_train = []
    #i = 0
    for idx in train_ids:
      #if i <= 1920:
        local_train.append(dataset[idx])
      #i += 1
    
    local_test = []
    #i = 0
    for idx in test_ids:
      #if i <= 480:
        local_test.append(dataset[idx])
      #i += 1

    training_args = TrainingArguments("trained_scibert_ner_model", # output dir
                                      learning_rate=7.648631456296879e-06, 
                                      weight_decay=0.4444157928489863,
                                      num_train_epochs=19, 
                                      dataloader_drop_last=True,
                                      per_device_eval_batch_size=8, 
                                      per_device_train_batch_size=8,
                                      logging_steps=50,
                                      save_steps=len(local_train) // batch_size,
                                      lr_scheduler_type='cosine',
                                      evaluation_strategy='steps',
                                      eval_steps=len(local_train) // batch_size
                                      )
    #learning_rate 7.648631456296879e-06
    #num_train_epochs 19
    #weight_decay 0.44441579284898636
    #per_device_train_batch_size 8

    # Init the neural network
    ner_model = NerModel(BertEmbModel).to('cuda')  # make sure we move the model to the GPU for training
    
    trainer = MultilabelTrainer(model=ner_model,
                                args=training_args,
                                train_dataset=local_train,
                                eval_dataset=local_test,
                                data_collator=collator  # defines how to merge data into batches, using the collator function above
                                )

    #Loading Best parameters
    #load_param()

    #Training
    trainer.train()
          
    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')
    
    # Saving the model
    save_path = f'./model-fold-{fold}.pth'
    torch.save(ner_model.state_dict(), save_path)

    # Evaluationfor this fold
    #correct, total = 0, 0
    with torch.no_grad():
      result = trainer.evaluate(local_test)
      print(result)

      # Print accuracy
      print('Accuracy for fold ', fold, ': ', result['eval_micro_f1_no_o'])
      print('--------------------------------')
      results[fold] = result['eval_micro_f1_no_o']
      resultss[fold] = result['eval_micro_f1']
      del result
    


FOLD 0
--------------------------------


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1606
  Num Epochs = 19
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3800


Step,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Micro F1,Micro Precision,Micro Recall,Macro F1 No O,Macro Precision No O,Macro Recall No O,Micro F1 No O,Micro Precision No O,Micro Recall No O
401,0.009700,0.038744,0.845879,0.784856,0.933405,0.942737,0.942737,0.942737,0.805170,0.715480,0.929788,0.812942,0.719924,0.933563
802,0.004900,0.021895,0.895151,0.854653,0.943002,0.964804,0.964804,0.964804,0.866527,0.808648,0.934121,0.874224,0.820457,0.935531
1203,0.002700,0.022043,0.884135,0.845315,0.932203,0.959148,0.959148,0.959148,0.852921,0.796762,0.921296,0.856166,0.797452,0.924213
1604,0.002400,0.021728,0.892430,0.883134,0.904459,0.967388,0.967388,0.967388,0.862054,0.849606,0.878145,0.869715,0.868861,0.870571
2005,0.001300,0.018123,0.893827,0.864173,0.928615,0.965503,0.965503,0.965503,0.864573,0.822452,0.913500,0.873325,0.835807,0.914370
2406,0.001600,0.018125,0.893173,0.866766,0.923543,0.963757,0.963757,0.963757,0.864011,0.825808,0.907444,0.867678,0.825122,0.914862
2807,0.001700,0.020007,0.890761,0.857275,0.930664,0.960335,0.960335,0.960335,0.861492,0.812539,0.918892,0.859361,0.801533,0.926181
3208,0.001100,0.017081,0.896352,0.871960,0.924358,0.965573,0.965573,0.965573,0.867885,0.832779,0.907771,0.872855,0.835358,0.913878
3609,0.001100,0.016901,0.900061,0.878667,0.924373,0.966550,0.966550,0.966550,0.872636,0.841718,0.907411,0.875737,0.840652,0.913878


Saving model checkpoint to trained_scibert_ner_model/checkpoint-401
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-802
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1203
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1604
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2005
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2406
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2807
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to tr

Training process has finished. Saving trained model.
Starting testing
{'eval_macro_f1': 0.8995894858631727, 'eval_macro_precision': 0.8778613590866896, 'eval_macro_recall': 0.9243327436723349, 'eval_micro_f1': 0.9664106145251397, 'eval_micro_precision': 0.9664106145251397, 'eval_micro_recall': 0.9664106145251397, 'eval_macro_f1_no_o': 0.8720353424111611, 'eval_macro_precision_no_o': 0.840644501024662, 'eval_macro_recall_no_o': 0.9074111061464465, 'eval_micro_f1_no_o': 0.8753240631628564, 'eval_micro_precision_no_o': 0.8398914518317503, 'eval_micro_recall_no_o': 0.9138779527559056, 'eval_loss': 0.016934644687280523, 'eval_runtime': 4.9843, 'eval_samples_per_second': 10.031, 'eval_steps_per_second': 1.404, 'epoch': 19.0}
Accuracy for fold  0 :  0.8753240631628564
--------------------------------
FOLD 1
--------------------------------


...

Training process has finished. Saving trained model.
Starting testing
{'eval_macro_f1': 0.9831296528527464, 'eval_macro_precision': 0.9786447502591404, 'eval_macro_recall': 0.9877365024833329, 'eval_micro_f1': 0.995395505463542, 'eval_micro_precision': 0.995395505463542, 'eval_micro_recall': 0.995395505463542, 'eval_macro_f1_no_o': 0.9783939620154255, 'eval_macro_precision_no_o': 0.9721098725770263, 'eval_macro_recall_no_o': 0.9848400917410812, 'eval_micro_f1_no_o': 0.9830164765525982, 'eval_micro_precision_no_o': 0.9773185483870968, 'eval_micro_recall_no_o': 0.9887812340642529, 'eval_loss': 0.002542914872610709, 'eval_runtime': 5.0006, 'eval_samples_per_second': 9.999, 'eval_steps_per_second': 1.4, 'epoch': 19.0}
Accuracy for fold  3 :  0.9830164765525982
--------------------------------
FOLD 4
--------------------------------


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1607
  Num Epochs = 19
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3800


Step,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Micro F1,Micro Precision,Micro Recall,Macro F1 No O,Macro Precision No O,Macro Recall No O,Micro F1 No O,Micro Precision No O,Micro Recall No O
401,0.006400,0.009797,0.988116,0.984581,0.991722,0.996415,0.996415,0.996415,0.984820,0.979761,0.989972,0.986350,0.979910,0.992875
802,0.003400,0.006331,0.985318,0.980972,0.989752,0.995174,0.995174,0.995174,0.981355,0.975135,0.987692,0.982332,0.974462,0.990331
1203,0.001900,0.004363,0.989631,0.989246,0.990071,0.996277,0.996277,0.996277,0.986892,0.986167,0.987692,0.986315,0.982332,0.990331
1604,0.001400,0.003496,0.989620,0.989053,0.990233,0.996277,0.996277,0.996277,0.986852,0.985884,0.987881,0.985808,0.981827,0.989822
2005,0.001100,0.002986,0.990817,0.991328,0.990381,0.996484,0.996484,0.996484,0.988395,0.988889,0.987999,0.986308,0.982820,0.989822
2406,0.000800,0.002710,0.991196,0.991892,0.990563,0.996484,0.996484,0.996484,0.988939,0.989722,0.988241,0.987059,0.984312,0.989822
2807,0.001500,0.002560,0.990147,0.989864,0.990483,0.996208,0.996208,0.996208,0.987594,0.987018,0.988241,0.986058,0.982323,0.989822
3208,0.001300,0.002487,0.990227,0.990507,0.990016,0.996208,0.996208,0.996208,0.987688,0.987875,0.987592,0.985801,0.982314,0.989313
3609,0.001000,0.002466,0.990579,0.990721,0.990503,0.996277,0.996277,0.996277,0.988156,0.988160,0.988241,0.986308,0.982820,0.989822


Saving model checkpoint to trained_scibert_ner_model/checkpoint-401
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-802
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1203
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1604
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2005
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2406
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2807
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to tr

Training process has finished. Saving trained model.
Starting testing
{'eval_macro_f1': 0.9905787393405279, 'eval_macro_precision': 0.9907206363454815, 'eval_macro_recall': 0.990502841355944, 'eval_micro_f1': 0.9962774024541569, 'eval_micro_precision': 0.9962774024541569, 'eval_micro_recall': 0.9962774024541569, 'eval_macro_f1_no_o': 0.9881563660331021, 'eval_macro_precision_no_o': 0.9881596989435989, 'eval_macro_recall_no_o': 0.9882408243303186, 'eval_micro_f1_no_o': 0.9863083164300204, 'eval_micro_precision_no_o': 0.9828196058615463, 'eval_micro_recall_no_o': 0.989821882951654, 'eval_loss': 0.0024650488651241174, 'eval_runtime': 4.9997, 'eval_samples_per_second': 10.001, 'eval_steps_per_second': 1.4, 'epoch': 19.0}
Accuracy for fold  4 :  0.9863083164300204
--------------------------------


# Results

In [ ]:
results

array([0.87532406, 0.97009654, 0.96940676, 0.98301648, 0.98630832])

In [ ]:
# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
sum = 0.0
key = 0
for value in results:
  print(f'Fold {key}: {value} %')
  sum += value
  key += 1
print(f'Average: {sum/len(results)} %')

K-FOLD CROSS VALIDATION RESULTS FOR 5 FOLDS
--------------------------------
Fold 0: 0.8753240631628564 %
Fold 1: 0.9700965387332234 %
Fold 2: 0.9694067571162543 %
Fold 3: 0.9830164765525982 %
Fold 4: 0.9863083164300204 %
Average: 0.9568304303989905 %


# Pytorch Training - Loop UPDT

In [22]:
device = 'cuda'

In [23]:
#'learning_rate': 7.416939461964081e-06, 'num_train_epochs': 12, 'weight_decay': 0.22055622812922648, 'per_device_train_batch_size': 8}

In [24]:
training_args = TrainingArguments("trained_scibert_ner_model", # output dir
                                      learning_rate=7.648631456296879e-06, 
                                      num_train_epochs=19, 
                                      dataloader_drop_last=True,
                                      per_device_eval_batch_size=8, 
                                      per_device_train_batch_size=8,
                                      logging_steps=50,
                                      save_steps=len(train) // batch_size,
                                      lr_scheduler_type='cosine',
                                      evaluation_strategy='steps',
                                      weight_decay=0.4444157928489863,
                                      eval_steps=len(train) // batch_size
                                      )

#learning_rate 7.648631456296879e-06
#num_train_epochs 19
#weight_decay 0.44441579284898636
#per_device_train_batch_size 8

#load_param()

In [25]:
loop_val = 30
loop_results = np.zeros(loop_val)
loop_resultss = np.zeros(loop_val)
for r in range(loop_val):

  ner_model = NerModel(BertEmbModel).to('cuda')  # make sure we move the model to the GPU for training

  trainer = MultilabelTrainer(
      model=ner_model, 
      args=training_args, 
      train_dataset=train, 
      eval_dataset=test,
      data_collator=collator  # defines how to merge data into batches, using the collator function above
  )

  # Print
  print(f'Train run #{r}')
  print('--------------------------------')

  trainer.train()

  # Process is complete.
  print('Training process has finished.')

  # Print about testing
  print('Starting testing')

  with torch.no_grad():
    result = trainer.evaluate(test)
    print(result)

    # Print accuracy
    print('Accuracy for fold ', r, ': ', result['eval_micro_f1_no_o'], ' -- ', result['eval_micro_f1'])
    print('--------------------------------')
    loop_results[r] = result['eval_micro_f1_no_o']
    loop_resultss[r] = result['eval_micro_f1']
    del result

  if r > 0:
    if loop_results[r] < loop_results[r-1]:
      save_path = f'./model-fold-{r}.pth'
      torch.save(ner_model.state_dict(), save_path)

  print('Testing process has finished.')

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1522
  Num Epochs = 19
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3610


Train run #0
--------------------------------


Step,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Micro F1,Micro Precision,Micro Recall,Macro F1 No O,Macro Precision No O,Macro Recall No O,Micro F1 No O,Micro Precision No O,Micro Recall No O
380,0.013600,0.031001,0.431051,0.499299,0.455481,0.865356,0.865356,0.865356,0.265358,0.375869,0.275602,0.215732,0.711160,0.127152
760,0.004700,0.025884,0.559517,0.716302,0.573667,0.886588,0.886588,0.886588,0.430383,0.652946,0.434467,0.396800,0.623116,0.291080
1140,0.002800,0.021995,0.618203,0.669953,0.665576,0.892262,0.892262,0.892262,0.506144,0.580084,0.564210,0.497202,0.552632,0.451878
1520,0.001800,0.022825,0.720182,0.734925,0.750691,0.918231,0.918231,0.918231,0.638750,0.661166,0.676620,0.638946,0.674196,0.607199
1900,0.002400,0.020452,0.813947,0.794510,0.843633,0.931918,0.931918,0.931918,0.763361,0.736352,0.804027,0.743954,0.730219,0.758216
2280,0.001700,0.020884,0.808956,0.782023,0.845339,0.929169,0.929169,0.929169,0.757421,0.719746,0.807677,0.740417,0.718865,0.763302
2660,0.001000,0.024039,0.815905,0.812345,0.823074,0.933848,0.933848,0.933848,0.765691,0.761958,0.774232,0.743825,0.757711,0.730438
3040,0.001400,0.021208,0.812136,0.788770,0.841759,0.932035,0.932035,0.932035,0.761035,0.728574,0.801827,0.746988,0.730640,0.764085
3420,0.001400,0.020971,0.813548,0.789555,0.844056,0.931976,0.931976,0.931976,0.762966,0.729559,0.805050,0.747853,0.730153,0.766432


Saving model checkpoint to trained_scibert_ner_model/checkpoint-380
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-760
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1140
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1520
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1900
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2280
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2660
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to tr

Training process has finished.
Starting testing


...

Step,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Micro F1,Micro Precision,Micro Recall,Macro F1 No O,Macro Precision No O,Macro Recall No O,Micro F1 No O,Micro Precision No O,Micro Recall No O
380,0.005200,0.051569,0.812816,0.787795,0.840943,0.930631,0.930631,0.930631,0.762639,0.726549,0.802824,0.754435,0.721686,0.790297
760,0.002600,0.054950,0.810347,0.790806,0.831736,0.929754,0.929754,0.929754,0.759469,0.731425,0.789953,0.748204,0.723848,0.774257
1140,0.001600,0.065089,0.810711,0.799399,0.824153,0.931274,0.931274,0.931274,0.759707,0.744315,0.777939,0.747860,0.743808,0.751956
1520,0.001100,0.055892,0.815915,0.787809,0.849218,0.927356,0.927356,0.927356,0.767693,0.726085,0.816127,0.753195,0.706023,0.807121
1900,0.000900,0.060644,0.815265,0.793792,0.839065,0.928175,0.928175,0.928175,0.766443,0.734884,0.801053,0.748185,0.713525,0.786385
2280,0.000700,0.053666,0.821586,0.803380,0.842107,0.931216,0.931216,0.931216,0.774306,0.747063,0.804583,0.758557,0.723050,0.797731
2660,0.000600,0.059726,0.815736,0.798739,0.834869,0.927999,0.927999,0.927999,0.767171,0.741705,0.795436,0.748228,0.714897,0.784820


Saving model checkpoint to trained_scibert_ner_model/checkpoint-380
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-760
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1140
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1520
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-1900
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2280
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-2660
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


Step,Training Loss,Validation Loss,Macro F1,Macro Precision,Macro Recall,Micro F1,Micro Precision,Micro Recall,Macro F1 No O,Macro Precision No O,Macro Recall No O,Micro F1 No O,Micro Precision No O,Micro Recall No O
380,0.005200,0.051569,0.812816,0.787795,0.840943,0.930631,0.930631,0.930631,0.762639,0.726549,0.802824,0.754435,0.721686,0.790297
760,0.002600,0.054950,0.810347,0.790806,0.831736,0.929754,0.929754,0.929754,0.759469,0.731425,0.789953,0.748204,0.723848,0.774257
1140,0.001600,0.065089,0.810711,0.799399,0.824153,0.931274,0.931274,0.931274,0.759707,0.744315,0.777939,0.747860,0.743808,0.751956
1520,0.001100,0.055892,0.815915,0.787809,0.849218,0.927356,0.927356,0.927356,0.767693,0.726085,0.816127,0.753195,0.706023,0.807121
1900,0.000900,0.060644,0.815265,0.793792,0.839065,0.928175,0.928175,0.928175,0.766443,0.734884,0.801053,0.748185,0.713525,0.786385
2280,0.000700,0.053666,0.821586,0.803380,0.842107,0.931216,0.931216,0.931216,0.774306,0.747063,0.804583,0.758557,0.723050,0.797731
2660,0.000600,0.059726,0.815736,0.798739,0.834869,0.927999,0.927999,0.927999,0.767171,0.741705,0.795436,0.748228,0.714897,0.784820
3040,0.000600,0.058787,0.822409,0.794988,0.854458,0.929227,0.929227,0.929227,0.775858,0.735028,0.822747,0.757515,0.708831,0.813380
3420,0.000600,0.059144,0.822423,0.797018,0.851967,0.929403,0.929403,0.929403,0.775832,0.737935,0.819150,0.757132,0.710852,0.809859


Saving model checkpoint to trained_scibert_ner_model/checkpoint-3040
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to trained_scibert_ner_model/checkpoint-3420
Trainer.model is not a `PreTrainedModel`, only saving its state dict.


Training completed. Do not forget to share your model on huggingface.co/models =)




Training process has finished.
Starting testing
{'eval_macro_f1': 0.8224626438905608, 'eval_macro_precision': 0.7972780260901976, 'eval_macro_recall': 0.851705388734413, 'eval_micro_f1': 0.9294613090015792, 'eval_micro_precision': 0.9294613090015792, 'eval_micro_recall': 0.9294613090015792, 'eval_macro_f1_no_o': 0.7758724307068081, 'eval_macro_precision_no_o': 0.7383028506876913, 'eval_macro_recall_no_o': 0.8187559826318318, 'eval_micro_f1_no_o': 0.7571820677035681, 'eval_micro_precision_no_o': 0.7112409762805088, 'eval_micro_recall_no_o': 0.8094679186228482, 'eval_loss': 0.05914935278548607, 'eval_runtime': 17.0396, 'eval_samples_per_second': 3.521, 'eval_steps_per_second': 0.469, 'epoch': 19.0}
Accuracy for fold  29 :  0.7571820677035681  --  0.9294613090015792
--------------------------------
Testing process has finished.


In [26]:
loop_results

array([0.74794573, 0.76577071, 0.76666667, 0.76928867, 0.77362719,
       0.77734448, 0.77256588, 0.76350093, 0.76414048, 0.7610226 ,
       0.77161863, 0.76669785, 0.76925905, 0.76301887, 0.76923077,
       0.76268657, 0.75323714, 0.7630447 , 0.76934636, 0.75754769,
       0.74713271, 0.74747475, 0.74859708, 0.7443595 , 0.75705152,
       0.76423991, 0.76517451, 0.75792237, 0.75293281, 0.75718207])

In [27]:
loop_resultss

array([0.93221033, 0.93343861, 0.93402351, 0.93612915, 0.93712347,
       0.93665555, 0.9358367 , 0.934082  , 0.93267825, 0.93267825,
       0.93519331, 0.93560274, 0.93402351, 0.9346669 , 0.93601217,
       0.93267825, 0.93086506, 0.93402351, 0.93554425, 0.93133298,
       0.92782359, 0.93039715, 0.93098204, 0.9306311 , 0.93390653,
       0.9346669 , 0.93571972, 0.93361408, 0.92636135, 0.92946131])

In [28]:
sum = 0.0
for value in loop_results:
  sum += value
print(f'Average micro_f1_no_o: {sum/len(loop_results)} %')

sum = 0.0
for value in loop_resultss:
  sum += value
print(f'Average micro_f1: {sum/len(loop_results)} %')

Average micro_f1_no_o: 0.7616542730424077 %
Average micro_f1: 0.9332787428593712 %


In [29]:
ner_model = NerModel(BertEmbModel).to('cuda')  # make sure we move the model to the GPU for training
ner_model.load_state_dict(torch.load(save_path))
ner_model.eval()

NerModel(
  (sci_embeddings): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

# Get Values of Thruth

In [30]:
# Pytorch thing (if we aren't training, do this)
ner_model.eval()
ner_model.to('cuda')

NerModel(
  (sci_embeddings): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [31]:
output_preds = []
output_real = []
for x in range(len(test)):
  inputs1 = test[x]['input_ids'][0].clone().detach().to(torch.long).unsqueeze(0).to('cuda')
  inputs2 = test[x]['attention_mask'][0].clone().detach().to(torch.long).unsqueeze(0).to('cuda')
  inputs = {'input_ids': inputs1,  'attention_mask': inputs2}
  #print(inputs)

  temp_test = test[x]
  temp_out = temp_test.pop("labels")
  output_real.append(np.array(temp_out[temp_out != -100])) 

  gen_preds = ner_model(**inputs)
  label_preds = np.argmax(gen_preds.cpu().detach().numpy(), axis=-1)[0]
  output_preds.append(label_preds[temp_out != -100])


In [ ]:
for x in range(len(output_real)):
  output_real[x] = [ID2Entity(y) for y in output_real[x]]
  output_preds[x] = [ID2Entity(z) for z in output_preds[x]]

In [ ]:
#output_real[28]

In [ ]:
from sklearn.metrics import mean_squared_error, multilabel_confusion_matrix, confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10, 7]
plt.rcParams["figure.autolayout"] = True
plt.rcParams.update({'font.size': 13})

labels = ["None", 'TASK', 'DATASET', 'METRIC']
cm = confusion_matrix(output_real[0], output_preds[0], labels=labels)
for x in range(len(output_real)-1):
  cm += confusion_matrix(output_real[x+1], output_preds[x+1], labels=labels)
print(cm)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(cmap=plt.cm.Blues)

In [ ]:
del cm

In [ ]:
len(output_real)

# Test over Text

In [ ]:
def prepare_input(txt):
  inputs = BertTokenizer(txt, return_tensors='pt', padding='max_length', truncation=True, max_length=150).to('cuda')
  return inputs

input_text = ["English is shown to be trans-context-free on the basis of coordinations of the respectively type that involve strictly syntactic cross-serial agreement."]
##input_text = ["The transient analysis of gyro-elastic structured media, composed of periodically placed masses interconnected by elastic rods and attached to gyroscopic spinners, is presented."] 
##input_text = ["The results indicated that thermal curing promoted the early strength of mortars, while decreased the late strength of mortars."] 
##input_text = ["A wide variety of processes are attested in the literature, and we find different forms of clippings in our data, including mixtures of different clippings, homophone respellings, phonetic respellings in-cluding informal oral forms, initialisms (but no acronyms), and mixtures of clipping together with homo-phone and phonetic respellings."] 

#input_text = ["The goal is to accurately predict the running time of applications for task scheduling and job migration."] 
#input_text = ["This paper reports on the development of a cross-domain framework for describing complex design practices."] 
#input_text = ["Studies of inequality in China typically ignore cost of liv-ing differences between areas."] 
#input_text = ["The present study was designed to explore the long-term differences be-tween three mouse models for depression."]
#input_text = ["Finally, regarding professional competencies, teachers appeared to be largely unprepared to conduct language assessments consistent with the LAR demands."] 

##input_text = ["propose a fast and reliable restoration method of virtual resources on OpenStack when physical servers or virtual machines are down."] 
##input_text = ["The results from our simulations reveal that the network assisted adaptation clearly outperforms the purely client-based DASH heuristics in some of the metrics, not all of them, particularly, in situations when the achievable throughput is moderately high or the link quality of the mobile clients does not differ from each other substantially."] 
##input_text = ["For hard rock drilling in coal mine, the drilling efficiency and service life of polycrystalline diamond compact bit are very low."] 
##input_text = ["Capturing changes in foreign reserves and exchange rates through the exchange market pressure, this article investigates whether economic policy uncertainty plays any role in exchange market pressure movements while controlling for the effects of domestic and external factors."] 
##input_text = ["This paper presents design of an self contained actuators unit in wide area damping control of power system in stabilizing system response for both nominal system condition and during actuator faults."] 

##input_text = ["Ultrasound-based brain stimulation techniques may become a powerful new technique to modulate the human brain in a focal and targeted manner."] 
#input_text = ["Recent work pre-training Transformers with self-supervised objectives on large text corpora has shown great success when fine-tuned on downstream NLP tasks including text summarization."]

# Tokenize + pad
inputs = prepare_input(input_text)

#inputs
#print(inputs)

In [ ]:
# Pytorch thing (if we aren't training, do this)
ner_model.eval()

# Get predictions
preds = ner_model(**inputs).cpu().detach().numpy()
preds = np.argmax(preds, axis=-1)[0]
pred_labels = [ID2Entity(x) for x in preds]

# Convert token ids to text
tokens = BertTokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

# Display result
for token, label in zip(tokens, pred_labels):
  if token == '[SEP]':
    break
  if token == '[CLS]':
    continue
  print('{} -> {}'.format(token, label))

a -> None
wide -> None
variety -> None
of -> None
processes -> None
are -> None
att -> None
##ested -> None
in -> None
the -> None
literature -> None
, -> None
and -> None
we -> None
find -> None
different -> None
forms -> None
of -> None
clip -> None
##ping -> None
##s -> None
in -> None
our -> None
data -> None
, -> None
including -> None
mixtures -> None
of -> None
different -> None
clip -> None
##ping -> None
##s -> None
, -> None
homo -> None
##phone -> None
resp -> None
##elling -> None
##s -> None
, -> None
phonetic -> None
resp -> None
##elling -> None
##s -> None
in -> None
- -> None
cl -> None
##uding -> None
informal -> None
oral -> None
forms -> None
, -> None
initial -> None
##isms -> None
( -> None
but -> None
no -> None
acr -> None
##onyms -> None
) -> None
, -> None
and -> None
mixtures -> None
of -> None
clip -> None
##ping -> None
together -> None
with -> None
homo -> None
- -> None
phone -> None
and -> None
phonetic -> None
resp -> None
##elling -> None
##s -> None
.

# Model Save and Load

In [ ]:
print("Our model: \n\n", ner_model, '\n')
print("The state dict keys: \n\n", ner_model.state_dict().keys())

Our model: 

 NerModel(
  (sci_embeddings): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [ ]:
torch.save(ner_model.state_dict(), 'trained_model_dic.pth')

In [ ]:
# download checkpoint file
files.download('trained_model_dic.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls

checkpoint.pth	  model-fold-1.pth  model-fold-4.pth  trained_model_dic.pth
dev.json	  model-fold-2.pth  sample_data       trained_scibert_ner_model
model-fold-0.pth  model-fold-3.pth  test.json	      train.json


Loading the model

In [ ]:
!ls

sample_data  test_500_v2.conll	train_1500_v2.conll  trained_model_dic.pth


In [ ]:
state_dict = torch.load('trained_model_dic.pth')
print(state_dict.keys())

odict_keys(['sci_embeddings.embeddings.position_ids', 'sci_embeddings.embeddings.word_embeddings.weight', 'sci_embeddings.embeddings.position_embeddings.weight', 'sci_embeddings.embeddings.token_type_embeddings.weight', 'sci_embeddings.embeddings.LayerNorm.weight', 'sci_embeddings.embeddings.LayerNorm.bias', 'sci_embeddings.encoder.layer.0.attention.self.query.weight', 'sci_embeddings.encoder.layer.0.attention.self.query.bias', 'sci_embeddings.encoder.layer.0.attention.self.key.weight', 'sci_embeddings.encoder.layer.0.attention.self.key.bias', 'sci_embeddings.encoder.layer.0.attention.self.value.weight', 'sci_embeddings.encoder.layer.0.attention.self.value.bias', 'sci_embeddings.encoder.layer.0.attention.output.dense.weight', 'sci_embeddings.encoder.layer.0.attention.output.dense.bias', 'sci_embeddings.encoder.layer.0.attention.output.LayerNorm.weight', 'sci_embeddings.encoder.layer.0.attention.output.LayerNorm.bias', 'sci_embeddings.encoder.layer.0.intermediate.dense.weight', 'sci_emb

In [ ]:
ner_model = NerModel(BertEmbModel).to('cuda')
ner_model.load_state_dict(state_dict)

<All keys matched successfully>

# Obtain datasets' weights values (Do not run - fixed values)

In [ ]:
zero = 0
one=0
two=0 
three=0
four=0
five=0
six=0
local_set = train
for i in range(len(local_set)):
  for j in range(len(local_set[i]['labels'])):
    if local_set[i]['labels'][j] == 0:
      zero += 1
    elif local_set[i]['labels'][j] == 1:
      one += 1
    elif local_set[i]['labels'][j] == 2:
      two += 1
    elif local_set[i]['labels'][j] == 3:
      three += 1
    elif local_set[i]['labels'][j] == 4:
      four += 1
    elif local_set[i]['labels'][j] == 5:
      five += 1
    elif local_set[i]['labels'][j] == 6:
      six += 1
    
print('0: ', zero, '1: ', one, '2: ', two, '3: ', three, '4: ', four, '5: ', five, '6: ', six)

0:  48170 1:  4364 2:  1859 3:  1133 4:  0 5:  0 6:  0


In [ ]:
zero = 0
one=0
two=0 
three=0
four=0
five=0
six=0
local_set = test
for i in range(len(local_set)):
  for j in range(len(local_set[i]['labels'])):
    if local_set[i]['labels'][j] == 0:
      zero += 1
    elif local_set[i]['labels'][j] == 1:
      one += 1
    elif local_set[i]['labels'][j] == 2:
      two += 1
    elif local_set[i]['labels'][j] == 3:
      three += 1
    elif local_set[i]['labels'][j] == 4:
      four += 1
    elif local_set[i]['labels'][j] == 5:
      five += 1
    elif local_set[i]['labels'][j] == 6:
      six += 1
    
print('0: ', zero, '1: ', one, '2: ', two, '3: ', three, '4: ', four, '5: ', five, '6: ', six)

0:  14719 1:  1337 2:  854 3:  386 4:  0 5:  0 6:  0
